**Instrucciones:**

Utiliza los conceptos aprendidos en los laboratorios de regresión y clasificación para encontrar el error estándar de los coeficientes de una regresión (lineal/logística) simple para los datasets de “Advertising” y “Default”.

Utiliza bootstrap para simular 1000 remuestreos de esos datasets y calcula la media de los coeficientes obtenidos al aplicarle regresión a cada remuestreo. Calcula la desviación estándar.

Compara los resultados obtenidos con el método visto en los laboratorios contra los resultados obtenidos con bootstrap. ¿Por qué podría haber diferencias en los resultados?

Agrega regularización L2 a los modelos del dataset de Advertising (optimiza el hiperparámetro). Utiliza ese valor del hiperparámetro para repetir el experimento de los 1000 remuestreos. Calcula la desviación estándar de los coeficientes obtenidos.

In [48]:
import numpy as np
import pandas as pd
from sklearn.utils import resample
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
import statsmodels.api as sm
import statsmodels.formula.api as smf

### ***ADVERTISING***

In [36]:
#Regresión Advertising

adv = pd.read_csv("Advertising.csv")

y = adv.iloc[:, -1]        #Sales (última columna)
X = adv.iloc[:, :-1]       #todas menos la última

X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

print(model.summary())

print("\nCoeficientes:")
print(model.params)

print("\nErrores estándar clásicos:")
print(model.bse)

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.895
Method:                 Least Squares   F-statistic:                     425.7
Date:                Thu, 20 Nov 2025   Prob (F-statistic):           3.94e-95
Time:                        23:14:01   Log-Likelihood:                -386.14
No. Observations:                 200   AIC:                             782.3
Df Residuals:                     195   BIC:                             798.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.0052      0.394      7.623      0.0

In [37]:
#Bootstrap (1000 remuestreos)

def bootstrap_ols(X, y, B=1000, seed=42):
    np.random.seed(seed)
    n = len(y)
    p = X.shape[1]
    coefs = []

    for b in range(B):
        idx = np.random.randint(0, n, n)
        X_b = X.iloc[idx]
        y_b = y.iloc[idx]

        model_b = sm.OLS(y_b, X_b).fit()
        coefs.append(model_b.params.values)

    return np.array(coefs)

In [43]:
#Bootstrap para Advertising (OLS simple)

B = 1000

coefs_boot_adv = bootstrap_ols(X_adv_sm, y_adv, B=B)

coef_boot_mean = coefs_boot_adv.mean(axis=0)
coef_boot_std  = coefs_boot_adv.std(axis=0, ddof=1)

print("Medias bootstrap:")
print(coef_boot_mean)

print("\nDesviaciones estándar bootstrap:")
print(coef_boot_std)

Medias bootstrap:
[7.0373413  0.04758717]

Desviaciones estándar bootstrap:
[0.34468534 0.00292386]


In [45]:
#COMPARACIÓN OLS vs BOOTSTRAP

comparison_adv = pd.DataFrame({
    "Coef_OLS": ols_adv.params.values,
    "SE_OLS": ols_adv.bse.values,
    "Coef_boot_mean": coef_boot_mean,
    "SE_bootstrap": coef_boot_std
}, index=ols_adv.params.index)

print(comparison_adv)

       Coef_OLS    SE_OLS  Coef_boot_mean  SE_bootstrap
const  7.032594  0.457843        7.037341      0.344685
TV     0.047537  0.002691        0.047587      0.002924


### Regularización L2 (Ridge) en Advertising + Bootstrap

In [40]:
#Ajustar Ridge y optimizar α

# X y y como arreglos
X_np = X_adv.values
y_np = y_adv.values

alphas = np.logspace(-4, 4, 80)

ridge = Ridge(fit_intercept=True)

grid = GridSearchCV(
    ridge,
    {'alpha': alphas},
    cv=5,
    scoring='neg_mean_squared_error'
)

grid.fit(X_np, y_np)

best_alpha = grid.best_params_['alpha']
print("\nMejor alpha (Ridge):", best_alpha)

ridge_best = Ridge(alpha=best_alpha, fit_intercept=True)
ridge_best.fit(X_np, y_np)

print("\nCoeficientes Ridge:")
print("Intercepto:", ridge_best.intercept_)
print("Coeficientes:", ridge_best.coef_)


Mejor alpha (Ridge): 10000.0

Coeficientes Ridge:
Intercepto: 7.079924375144942
Coeficientes: [0.04721476]


In [46]:
#Bootstrap para Ridge (1000 remuestreos)

def bootstrap_ridge(X, y, alpha, B=1000, seed=42):
    np.random.seed(seed)
    n = len(y)
    coefs = []

    for b in range(B):
        idx = np.random.randint(0, n, n)
        X_b = X[idx]
        y_b = y[idx]

        model = Ridge(alpha=alpha, fit_intercept=True)
        model.fit(X_b, y_b)

        params = np.concatenate([[model.intercept_], model.coef_])
        coefs.append(params)

    return np.array(coefs)

coefs_ridge_boot = bootstrap_ridge(X_np, y_np, alpha=best_alpha)

ridge_mean = coefs_ridge_boot.mean(axis=0)
ridge_std  = coefs_ridge_boot.std(axis=0, ddof=1)

print("Medias:", ridge_mean)
print("Desv. estándar:", ridge_std)

Medias: [7.0851266  0.04726169]
Desv. estándar: [0.34301635 0.00290554]


In [47]:
#Comparación OLS vs Ridge (Bootstrap)

cols = ['const'] + list(X_adv.columns)

comparison_final = pd.DataFrame({
    "OLS_mean": coef_boot_mean,
    "OLS_std": coef_boot_std,
    "Ridge_mean": ridge_mean,
    "Ridge_std": ridge_std
}, index=cols)

print(comparison_final)

       OLS_mean   OLS_std  Ridge_mean  Ridge_std
const  7.037341  0.344685    7.085127   0.343016
TV     0.047587  0.002924    0.047262   0.002906


**Conclusión:**

Ridge y OLS dan casi los mismos resultados porque el dataset está bien hecho y no tiene problemas.
La única diferencia es que Ridge hace que los coeficientes varíen un poquito menos, o sea, son más estables cuando cambias la muestra

### ***DEFAULT***

In [49]:
df = pd.read_csv("Default.csv")

# Crear variable binaria: 1 = 'Yes', 0 = 'No'
df["default_bin"] = (df["default"] == "Yes").astype(int)

print(df.head())
print(df["default_bin"].value_counts())

  default student      balance        income  default_bin
0      No      No   729.526495  44361.625074            0
1      No     Yes   817.180407  12106.134700            0
2      No      No  1073.549164  31767.138950            0
3      No      No   529.250605  35704.493940            0
4      No      No   785.655883  38463.495880            0
default_bin
0    9667
1     333
Name: count, dtype: int64


In [55]:
# Modelo logístico simple default ~ balance

logit_model = smf.logit("default_bin ~ balance", data=df).fit()
print(logit_model.summary())

Optimization terminated successfully.
         Current function value: 0.079823
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:            default_bin   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9998
Method:                           MLE   Df Model:                            1
Date:                Thu, 20 Nov 2025   Pseudo R-squ.:                  0.4534
Time:                        23:26:09   Log-Likelihood:                -798.23
converged:                       True   LL-Null:                       -1460.3
Covariance Type:            nonrobust   LLR p-value:                6.233e-290
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -10.6513      0.361    -29.491      0.000     -11.359      -9.943
balance        0.0055      0

In [56]:
#Errores

coef_analiticos = logit_model.params        # β0 y β1
se_analiticos   = logit_model.bse           # errores estándar

print("Coeficientes (método analítico):")
print(coef_analiticos)

print("\nErrores estándar (método analítico):")
print(se_analiticos)

Coeficientes (método analítico):
Intercept   -10.651331
balance       0.005499
dtype: float64

Errores estándar (método analítico):
Intercept    0.361169
balance      0.000220
dtype: float64


In [57]:
#Bootstrap con 1000 remuestreos

B = 1000
coefs_boot = []

for b in range(B):
    #Remuestreo con reemplazo
    sample = df.sample(n=len(df), replace=True)

    #Ajustar el mismo modelo logístico
    try:
        m_boot = smf.logit("default_bin ~ balance", data=sample).fit(disp=False)
        coefs_boot.append(m_boot.params.values)  # [intercepto, beta_balance]
    except:
        continue

coefs_boot = np.array(coefs_boot)
print("Número efectivo de boots:", coefs_boot.shape[0])

#Media de los coeficientes
boot_mean = coefs_boot.mean(axis=0)

#Desviación estándar (error estándar bootstrap)
boot_se = coefs_boot.std(axis=0, ddof=1)

print("\nMedia bootstrap de los coeficientes [intercepto, balance]:")
print(boot_mean)

print("\nErrores estándar bootstrap [intercepto, balance]:")
print(boot_se)

Número efectivo de boots: 1000

Media bootstrap de los coeficientes [intercepto, balance]:
[-1.06554884e+01  5.50143306e-03]

Errores estándar bootstrap [intercepto, balance]:
[3.64195541e-01 2.20913682e-04]


In [58]:
#Comparación Método analítico vs Bootstrap

import pandas as pd

tabla = pd.DataFrame({
    "Coeficiente": ["Intercepto", "Balance"],
    "SE Analítico": [se_analiticos[0], se_analiticos[1]],
    "SE Bootstrap": [boot_se[0], boot_se[1]]
})

tabla

/tmp/ipython-input-2707971881.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "SE Analítico": [se_analiticos[0], se_analiticos[1]],


,Coeficiente,SE Analítico,SE Bootstrap
0,Intercepto,0.361169,0.364196
1,Balance,0.000220,0.000221


**Conclusión:**

Los errores estándar obtenidos por el método analítico y los obtenidos mediante bootstrap con 1000 remuestreos son prácticamente iguales.

La diferencia es muy pequeña (mínima), lo cual nos puede decir que:

- El modelo logístico está bien especificado

- El bootstrap no revela variabilidad adicional relevante